In [51]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))


In [52]:
# para actualizar cambios del .py en el IPython notebook
%reload_ext autoreload
%autoreload 2

In [53]:
from data import load_last_12_months_data


# descargamos todo el año 2024
rides_all = load_last_12_months_data(datetime(2025, 1, 1), months_ago=12)
rides_all['pickup_datetime'].describe()

Descargando datos de 2024-01
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2024_01.parquet
Datos cargados desde ../data/raw/rides_2024_01.parquet
Descargando datos de 2024-02
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2024_02.parquet
Datos cargados desde ../data/raw/rides_2024_02.parquet
Descargando datos de 2024-03
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2024_03.parquet
Datos cargados desde ../data/raw/rides_2024_03.parquet
Descargando datos de 2024-04
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2024_04.parquet
Datos cargados desde ../data/raw/rides_2024_04.parquet
Descargando datos de 2024-05
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2024_05.parquet
Datos cargados desde ../data/raw/rides_2

count                      41169300
mean     2024-07-06 10:09:33.440509
min             2024-01-01 00:00:00
25%             2024-04-06 20:07:18
50%      2024-07-03 23:34:21.500000
75%             2024-10-08 17:33:34
max             2024-12-31 23:59:58
Name: pickup_datetime, dtype: object

In [54]:
from data import transform_to_time_series

# Transformamos el dataframe a una serie temporal
ts_data = transform_to_time_series(rides_all)
ts_data

/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:148: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
Rellenando huecos por localización: 100%|██████████| 263/263 [00:00<00:00, 437.67it/s]


,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,4,25
1,2024-01-01 01:00:00,4,29
2,2024-01-01 02:00:00,4,34
3,2024-01-01 03:00:00,4,31
4,2024-01-01 04:00:00,4,32
...,...,...,...
2310187,2024-12-31 19:00:00,110,0
2310188,2024-12-31 20:00:00,110,0
2310189,2024-12-31 21:00:00,110,0
2310190,2024-12-31 22:00:00,110,0


In [55]:
from data import transform_to_features_and_target

# Transformamos la serie temporal a un dataframe de características y objetivo
location_id = 138 # zona con id 138
n_lags = 24*28 # 4 semanas lags

X, y, df_location = transform_to_features_and_target(ts_data,n_lags = 24*28, location_id = 138)
df_location


/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,pickup_hour,pickup_location_id,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,...,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,target
0,2024-01-29 00:00:00,138,59.0,4.0,0.0,0.0,0.0,1.0,1.0,8.0,...,207.0,256.0,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138
1,2024-01-29 01:00:00,138,4.0,0.0,0.0,0.0,1.0,1.0,8.0,64.0,...,256.0,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15
2,2024-01-29 02:00:00,138,0.0,0.0,0.0,1.0,1.0,8.0,64.0,139.0,...,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25
3,2024-01-29 03:00:00,138,0.0,0.0,1.0,1.0,8.0,64.0,139.0,199.0,...,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25.0,2
4,2024-01-29 04:00:00,138,0.0,1.0,1.0,8.0,64.0,139.0,199.0,138.0,...,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8107,2024-12-31 19:00:00,138,330.0,328.0,310.0,366.0,381.0,71.0,16.0,0.0,...,142.0,160.0,117.0,87.0,173.0,113.0,221.0,160.0,137.0,64
8108,2024-12-31 20:00:00,138,328.0,310.0,366.0,381.0,71.0,16.0,0.0,0.0,...,160.0,117.0,87.0,173.0,113.0,221.0,160.0,137.0,64.0,53
8109,2024-12-31 21:00:00,138,310.0,366.0,381.0,71.0,16.0,0.0,0.0,3.0,...,117.0,87.0,173.0,113.0,221.0,160.0,137.0,64.0,53.0,100
8110,2024-12-31 22:00:00,138,366.0,381.0,71.0,16.0,0.0,0.0,3.0,6.0,...,87.0,173.0,113.0,221.0,160.0,137.0,64.0,53.0,100.0,63


In [41]:
# guardamos datos
from paths import TRANSFORMED_DATA_DIR

df_location.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_year_2024_4weeks_lags.parquet')